# Crawler for centaur.delegateselect.com

- [Customer search  page](https://centaur.delegateselect.com/MT/Run/event/main.html;jsessionid=9C11F56845D0A7C1C25E50152958BC99?Menu=attendees&Action=Start)

In [5]:
#Configs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
# imports
import requests
from bs4 import BeautifulSoup

In [40]:
# session id is refreshed every x minutes and is required to make requests
session_id = '84E96567119D41B50C38E62F6BEAE8DB'

# index start for the result payload(always 15 results)
company_index = 1

# when the sessionid is expired, a request will return:
# b'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'
url =  f'https://centaur.delegateselect.com/MT/Run/event/attendees.html;jsessionid={ session_id }?AJAX=1&PSVal=1'

In [43]:
# site content with the first 15 companies payload
site_content = requests.get(url).content

In [46]:
soup = BeautifulSoup(site_content)

In [50]:
soup.